<a href="https://colab.research.google.com/github/Kurumi0729/japan-e-stat-api-sample/blob/main/e_stat_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests 
import numpy as np
import pandas as pd

application_id = 'this is your application id'


####input the endpoint (you can get this on the e-stat page)
url = 'http://api.e-stat.go.jp/rest/3.0/app/getStatsData' 

params = {'appId':application_id,
        'lang':'J',
        'statsDataId':'0003449073',
        'metaGetFlg':'Y',
        'cntGetFlg':'N',
        'explanationGetFlg':'Y',
        'annotationGetFlg':'Y',
        'sectionHeaderFlg':'1',
        'replaceSpChars':'0'} #パラメータ

res = requests.get(url, params=params)

In [2]:
import defusedxml.ElementTree as ET

def get_estat_data():
  tree = ET.fromstring(res.text)

  dict_array = []
  for i in tree.iter(tag='VALUE'):
    tmp_array = i.attrib
    tmp_array['val'] = i.text
    dict_array.append(tmp_array)

  result_df = pd.DataFrame(dict_array)
  for i in tree.iter(tag='CLASS_OBJ'):
    code_l = []
    name_l = []
    level_l = []
    unit_l = []
    for s in i.findall('CLASS'):
      code_l.append(s.attrib['code'])
      name_l.append(s.attrib['name'])
    tmp_df = pd.DataFrame({i.attrib['id']:code_l, i.attrib['name']:name_l}) 

    result_df = pd.merge(result_df, tmp_df, on=i.attrib['id'])

  return result_df

df = get_estat_data()

In [3]:
df.head()

,tab,cat01,cat02,cat03,time,unit,val,表章項目,在留資格審査の受理・処理,在留資格審査,地方出入国在留管理局・支局,時間軸（月次）
0,160,100000,10,100000,2023000101,人,1706,在留資格取得等処理人員,受理_総数,資格取得,総数,2023年1月
1,160,102000,10,100000,2023000101,人,348,在留資格取得等処理人員,受理_旧受,資格取得,総数,2023年1月
2,160,103000,10,100000,2023000101,人,1358,在留資格取得等処理人員,受理_新受,資格取得,総数,2023年1月
3,160,300000,10,100000,2023000101,人,1363,在留資格取得等処理人員,既済_総数,資格取得,総数,2023年1月
4,160,301000,10,100000,2023000101,人,1265,在留資格取得等処理人員,既済_許可,資格取得,総数,2023年1月


In [5]:
df_test = df.query(f'在留資格審査 == "永住"')

In [6]:
df_test = df_test[['val', '表章項目','在留資格審査の受理・処理', '在留資格審査', '地方出入国在留管理局・支局', '時間軸（月次）']]
df_test.columns = ['val','big_category','status','category','group','time']
df_test.head()

,val,big_category,status,category,group,time
35,34265,在留資格取得等処理人員,受理_総数,永住,総数,2023年1月
36,30015,在留資格取得等処理人員,受理_旧受,永住,総数,2023年1月
37,4250,在留資格取得等処理人員,受理_新受,永住,総数,2023年1月
38,5203,在留資格取得等処理人員,既済_総数,永住,総数,2023年1月
39,3338,在留資格取得等処理人員,既済_許可,永住,総数,2023年1月


In [7]:
status = ["受理_総数", "受理_旧受", "受理_新受", "既済_総数", "既済_許可", "既済_不許可", "既済_その他"]
df_all = pd.DataFrame()
for i in status:
  df_data = df_test.query(f'group == "総数" and status == "{i}"')
  df_all[i] = df_data['val'].to_list()

In [8]:
df_all['time'] = df_data['time'].to_list()
df_all.head()
for i in df_all.columns:
  if i == 'time':
    pass
  else:
    df_all[i] = df_all[i].astype('int64')

In [17]:
df_all.columns = ['all','old','new','finish','accept','not_accept','else','time',]
df_all.head()

,all,old,new,finish,accept,not_accept,else,time
0,34265,30015,4250,5203,3338,1725,140,2023年1月
1,35853,31251,4602,5837,4019,1680,138,2022年12月
2,36823,31827,4996,5571,3496,1889,186,2022年11月
3,36616,31116,5500,4788,3105,1535,148,2022年10月
4,36140,30310,5830,5017,3233,1642,142,2022年9月


In [13]:
from google.cloud import bigquery
project_id = ""
client = bigquery.Client(project = project_id)
table_id = "estat.visa"

In [18]:
from google.colab import auth
auth.authenticate_user()

job_config = bigquery.LoadJobConfig(write_disposition = bigquery.WriteDisposition.WRITE_APPEND)
job = client.load_table_from_dataframe(df_all,table_id,job_config = job_config)